## ***ITERATIVE WORKFLOW***

### 🧩 ***Problem Statement***

Creating high-quality content consistently for platforms like LinkedIn and Instagram is time-consuming and subjective. Automating this process requires a robust system that ensures the content is relevant, engaging, and well-aligned with platform norms.


### 🔄 Proposed Solution: Content Generation Workflow

We'll design a modular, automated workflow that transforms a topic into platform-ready content. The workflow follows these steps:

- Input Topic
The user provides a brief topic or prompt (e.g., “AI and mental health”).

- Content Generation Pipeline
The workflow—built using LangChain and integrated tools—generates a polished post using templates, embeddings, and style tuning suited for each platform.

- Platform Deployment
Finalized content is automatically scheduled or posted via API integrations for LinkedIn, Instagram, etc.


```text
graph TD
    A[Input Topic] --> B[Generate Content]
    B --> C[Evaluate Quality]
    C -->|Approved| D[Deploy to Platform]
    C -->|Needs Improvement| E[Optimiser Refines Post]
    E --> B
```

In [21]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI

from langchain_core.messages import SystemMessage, HumanMessage

import operator


In [22]:
generator_llm = ChatOpenAI(model ='gpt-4o')
evaluator_llm = ChatOpenAI(model ='gpt-4o-mini')
optimizer_llm = ChatOpenAI(model ='gpt-4o')

In [23]:
# state 

class TweetState(TypedDict):
    topic : str
    tweet : str
    evaluation : Literal["approved", "needs_improvement"]
    feedback : str
    iteration : int
    max_iterations : int
    tweet_history : Annotated[list[str],operator.add]
    feedback_history :Annotated[list[str],operator.add]


In [24]:
from pydantic import BaseModel, Field
class TweetEvaluation(BaseModel):
    evaluation: Literal['approved', 'needs_improvement'] = Field(..., description="Final evaluation output")
    feedback: str = Field(..., description="Feedback for the tweet")

In [25]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [39]:
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day English.
""")
    ]
    response = generator_llm.invoke(messages).content
    return {'tweet': response, "tweet_history": [response]}

# --- Node 2: Evaluation ---
def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="""
You are a ruthless and dry-humored Twitter critic. You do not laugh, empathize, or offer encouragement.
Your job is to evaluate tweets solely for performance and quality. You reject tweets without hesitation.
"""),
        HumanMessage(content=f"""Evaluate the following tweet:
Tweet: {state['tweet']}

Strictly evaluate using these criteria:
- Originality: is this fresh or overused?
- Humor (if intended)
- Punchiness
- Virality Potential
- Format Compliance:
    - REJECT if Q&A format
    - REJECT if exceeds 280 characters
    - REJECT if ends with vague statements like “Let that sink in.”
- ⚖️ Bias Neutrality: REJECT if overly opinionated

Respond ONLY in this format:
Evaluation: "approved" or "needs_improvement"
Feedback: <feedback>
""")
    ]
    response = structured_evaluator_llm.invoke(messages)
    return {
        'evaluation': response.evaluation,
        'feedback': response.feedback,
        'feedback_history': [response.feedback]
    }

# --- Node 3: Optimizer ---
def optimize_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on the feedback: "{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet: "{state['tweet']}"

Make it as short as possible, under 200 characters. No Q&A format.
""")
    ]
    response = optimizer_llm.invoke(messages).content
    return {
        'tweet': response,
        "tweet_history": [response],
        'iteration': state['iteration'] + 1
    }

# --- Routing function ---
def route_evaluation(state: TweetState):
    if state['evaluation'] == "approved" or state['iteration'] >= state['max_iterations']:
        return "approved"
    else:
        return "needs_improvement" 


In [40]:
graph = StateGraph(TweetState)

graph.add_node("generate", generate_tweet)
graph.add_node("evaluate", evaluate_tweet)
graph.add_node("optimize", optimize_tweet)

graph.add_edge(START, "generate")
graph.add_edge("generate", "evaluate")
graph.add_conditional_edges("evaluate", route_evaluation, {
    "approved": END,
    "needs_improvement": "optimize"
})
graph.add_edge("optimize", "evaluate")

# Compile and run
workflow = graph.compile()

In [41]:
workflow = graph.compile()

In [42]:
initial_state = {
    "topic": "indian national flag",
    "max_iterations": 4,
    "iteration": 1,
    "tweet": "",
    "evaluation": "needs_improvement",
    "feedback": ""
}

workflow.invoke(initial_state)


{'topic': 'indian national flag',
 'tweet': 'Forget coffee, my morning routine: salute the Indian flag and reset the Wi-Fi. 🇮🇳 #PatrioticPriorities',
 'evaluation': 'needs_improvement',
 'feedback': "The tweet suffers from a lack of originality, as the concept of prioritizing items in morning routines is overused. The humor attempts are mild at best, and it lacks punchiness. The combination of flag saluting and Wi-Fi resetting feels forced and lacks a cohesive theme, diminishing its virality potential. Overall, it's a mediocre attempt that fails to resonate.",
 'iteration': 4,
 'max_iterations': 4,
 'tweet_history': ["When I see the Indian flag waving proudly, it reminds me of my Wi-Fi—looks strong, but I always pray it doesn't suddenly crash during important moments. 🇮🇳 #TricolorTales",
  "Seeing the Indian flag wave is like my Wi-Fi—majestic from afar, but closer inspection reveals a signal I pray won't drop during crucial moments. 🇮🇳 #TricolorTruth",
  'The Indian flag: a symbol of 

In [ ]:
initial_state = {
    "topic": "Donald trump",
    "max_iterations": 4,
    "iteration": 1,
    "tweet": "",
    "evaluation": "needs_improvement",
    "feedback": ""
}

workflow.invoke(initial_state)


{'topic': 'Donald trump',
 'tweet': '"New Trump trading cards: brimming with powers, deals, and debts! Watch out, these aren\'t your usual collectibles—saddle up for a political adventure! #TrumpCards"',
 'evaluation': 'needs_improvement',
 'feedback': "The tweet attempts to be catchy but falls flat on originality; the concept of trading cards featuring politicians is not fresh. It's also overly wordy without delivering a clear punch. The humor is forced and lacks real wit. The virality potential is low, as it doesn't offer a compelling hook or engaging insight. It leans toward opinionated language like 'political adventure,' which may alienate neutral viewers.",
 'iteration': 4,
 'max_iterations': 4}